This is a tutorial notebook on Riemannian optimization for machine learning, prepared for the Machine Learning Summer School 2019 (MLSS-2019, http://mlss2019.skoltech.ru) in Moscow, Russia, Skoltech (http://skoltech.ru).

Copyright 2019 by Alexey Artemov and ADASE 3DDL Team. Special thanks to Alexey Zaytsev for a valuable contribution.

## Index

1. [Generate a toy dataset](#Generate-a-toy-dataset).
2. [Use Riemannian optimization to obtain GMM estimates](#Use-Riemannian-optimization-to-obtain-GMM-estimates).
3. [GMM with real-world data using Riemannian optimization](#GMM-with-real-world-data-using-Riemannian-optimization).

## Riemannian Optimisation with `pymanopt` for inference in Gaussian mixture models

This notebook is the second in the series of two notebooks on Riemannian optimization and is based heavily on the [official mixture of Gaussian notebook](https://github.com/pymanopt/pymanopt/blob/master/examples/MoG.ipynb) from `pymanopt` docs. 

For the basic introduction, see the first part `riemannian_opt_for_ml.ipynb`.

Install the necessary libraries

In [0]:
!pip install --upgrade git+https://github.com/mlss-skoltech/tutorials.git#subdirectory=geometric_techniques_in_ML

In [0]:
!pip install pymanopt autograd
!pip install scipy==1.2.1 -U

In [0]:
import pkg_resources

DATA_PATH = pkg_resources.resource_filename('riemannianoptimization', 'data/')

### Generate a toy dataset

The Mixture of Gaussians (MoG) model assumes that datapoints $\mathbf{x}_i\in\mathbb{R}^d$ follow a distribution described by the following probability density function:
$$
p(\mathbf{x}) = \sum_{m=1}^M \pi_m p_\mathcal{N}(\mathbf{x};\mathbf{\mu}_m,\mathbf{\Sigma}_m)
$$ 

where $\pi_m$ is the probability that the data point belongs to the $m^\text{th}$ mixture component and $p_\mathcal{N}(\mathbf{x};\mathbf{\mu}_m,\mathbf{\Sigma}_m)$ is the probability density function of a [multivariate Gaussian distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution) with mean $\mathbf{\mu}_m \in \mathbb{R}^d$ and [positive semi-definite](https://en.wikipedia.org/wiki/Definiteness_of_a_matrix) (PSD) covariance matrix $\mathbf{\Sigma}_m \in \{\mathbf{M}\in\mathbb{R}^{d\times d}: \mathbf{M}\succeq 0\}$.

As an example consider the mixture of three Gaussians with means
$$
\mathbf{\mu}_1 = \begin{bmatrix} -4 \\ 1 \end{bmatrix},
\quad
\mathbf{\mu}_2 = \begin{bmatrix} 0 \\ 0 \end{bmatrix},
\quad
\mathbf{\mu}_3 = \begin{bmatrix} 2 \\ -1 \end{bmatrix},
$$
covariances
$$\mathbf{\Sigma}_1 = \begin{bmatrix} 3 & 0 \\ 0 & 1 \end{bmatrix},
\mathbf{\Sigma}_2 = \begin{bmatrix} 1 & 1 \\ 1 & 3 \end{bmatrix},
\mathbf{\Sigma}_3 = \begin{bmatrix} 0.5 & 0 \\ 0 & 0.5 \end{bmatrix}$$
and mixture probability vector $\pi=\left[0.1, 0.6, 0.3\right]$.
Let's generate $N=1000$ samples of that MoG model and scatter plot the samples:

Generate a synthetic dataset of $M=3$ Gaussian distributions, w

In [0]:
import numpy as np
np.set_printoptions(precision=2)

toy_n_points = 1000  # Number of data
toy_dim = 2  # Dimension of data
toy_components = 3  # Number of clusters 

# mixture parameters
toy_pi = [0.1, 0.6, 0.3]
toy_mus = [np.array([-4, 1]),
           np.array([0, 0]),
           np.array([2, -1])]
toy_sigmas = [np.array([[3, 0],[0, 1]]),
              np.array([[1, 1.], [1, 3]]),
              .5 * np.eye(2)]

# select which component work in each case
components = np.random.choice(toy_components, size=toy_n_points, p=toy_pi)

# prepare data
samples = np.zeros((toy_n_points, toy_dim))

# for each component, generate all needed samples
for k in range(toy_components):
    # indices of current component in X
    indices = (k == components)
    # number of those occurrences
    n_k = indices.sum()
    if n_k > 0:
        samples[indices] = np.random.multivariate_normal(toy_mus[k], toy_sigmas[k], n_k)

The following is a bunch of helper functions for visualizations.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm # Colormaps


def multivariate_normal(x, d, mean, covariance):
    """pdf of the multivariate normal distribution."""
    x_m = x - mean
    pdf = (1. / (np.sqrt((2 * np.pi)**d * np.linalg.det(covariance))) * 
            np.exp(-(np.linalg.solve(covariance, x_m).T.dot(x_m)) / 2))
    return pdf


# Plot bivariate distribution
def generate_surface(mean, covariance, d):
    """Helper function to generate density surface."""
    nb_of_x = 100 # grid size
    # choose limits adaptively
#     mu1, mu2 = mean[:, 0]
#     sigmasq1, sigmasq2 = covariance[0, 0], covariance[1, 1]
#     min_x1 = mu1 - 3. * np.sqrt(sigmasq1)
#     max_x1 = mu1 + 3. * np.sqrt(sigmasq1)
#     min_x2 = mu2 - 3. * np.sqrt(sigmasq2)
#     max_x2 = mu2 + 3. * np.sqrt(sigmasq2)
#     print(min_x1, max_x1)
#     print(min_x2, max_x2)
    min_x1, max_x1 = -4, 4
    min_x2, max_x2 = -4, 4
    x1s = np.linspace(min_x1, max_x1, num=nb_of_x)
    x2s = np.linspace(min_x2, max_x2, num=nb_of_x)
    x1, x2 = np.meshgrid(x1s, x2s) # Generate grid
    pdf = np.zeros((nb_of_x, nb_of_x))
    
    # Fill the cost matrix for each combination of weights
    for i in range(nb_of_x):
        for j in range(nb_of_x):
            pdf[i,j] = multivariate_normal(
                np.matrix([[x1[i,j]], [x2[i,j]]]), 
                d, mean, covariance)
    return x1, x2, pdf  # x1, x2, pdf(x1,x2)


def plot_gaussian(mu, sigma, ax):
    bivariate_mean = np.matrix(mu)  # Mean
    bivariate_covariance = np.matrix(sigma)  # Covariance
    x1, x2, p = generate_surface(
        bivariate_mean, bivariate_covariance, d=2)    
    # Plot bivariate distribution
    con = ax.contour(x1, x2, p, 10, cmap=cm.hot)
    # ax2.axis([-2.5, 2.5, -1.5, 3.5])
    ax.set_aspect('equal')

In [0]:
fig = plt.figure(figsize=(8,8))
ax = fig.gca()

for mu, sigma in zip(toy_mus, toy_sigmas):
    mu = np.matrix(mu).T  # plot_gaussian requires mu to be a column vector
    plot_gaussian(mu, sigma, ax)

colors = ['r', 'g', 'b', 'c', 'm']
for i in range(toy_components):
    indices = (i == components)
    ax.scatter(samples[indices, 0], samples[indices, 1], alpha=.4, color=colors[i % toy_components])


### Use Riemannian optimization to obtain GMM estimates

Given a data sample the de facto standard method to infer the parameters is the [expectation maximisation](https://en.wikipedia.org/wiki/Expectation-maximization_algorithm) (EM) algorithm that, in alternating so-called E and M steps, maximises the log-likelihood of the data.

In [arXiv:1506.07677](http://arxiv.org/pdf/1506.07677v1.pdf) Hosseini and Sra propose Riemannian optimisation as a powerful counterpart to EM. Importantly, they introduce a reparameterisation that leaves local optima of the log-likelihood unchanged while resulting in a geodesically convex optimisation problem over a product manifold $\prod_{m=1}^M\mathcal{PD}^{(d+1)\times(d+1)}$ of manifolds of $(d+1)\times(d+1)$ positive definite matrices.
The proposed method is on par with EM and shows less variability in running times.

The reparameterised optimisation problem for augmented data points $\mathbf{y}_i=[\mathbf{x}_i\ 1]$ can be stated as follows:

$$\min_{(S_1, ..., S_m, \nu_1, ..., \nu_{m-1}) \in \prod_{m=1}^M \mathcal{PD}^{(d+1)\times(d+1)}\times\mathbb{R}^{M-1}}
-\sum_{n=1}^N\log\left(
\sum_{m=1}^M \frac{\exp(\nu_m)}{\sum_{k=1}^M\exp(\nu_k)}
q_\mathcal{N}(\mathbf{y}_n;\mathbf{S}_m)
\right)$$

where

* $\mathcal{PD}^{(d+1)\times(d+1)}$ is the manifold of positive definite
$(d+1)\times(d+1)$ matrices
* $\mathcal{\nu}_m = \log\left(\frac{\alpha_m}{\alpha_M}\right), \ m=1, ..., M-1$ and $\nu_M=0$
* $q_\mathcal{N}(\mathbf{y}_n;\mathbf{S}_m) =
2\pi\exp\left(\frac{1}{2}\right)
|\operatorname{det}(\mathbf{S}_m)|^{-\frac{1}{2}}(2\pi)^{-\frac{d+1}{2}}
\exp\left(-\frac{1}{2}\mathbf{y}_i^\top\mathbf{S}_m^{-1}\mathbf{y}_i\right)$

**Optimisation problems like this can easily be solved using Pymanopt – even without the need to differentiate the cost function manually!**

So let's infer the parameters of our toy example by Riemannian optimisation using Pymanopt:

In [0]:
import pymanopt as opt
import pymanopt.solvers as solvers
import pymanopt.manifolds as manifolds

In [0]:
import autograd.numpy as np  # import here to avoid errors
from autograd.scipy.misc import logsumexp

# (1) Instantiate the manifold
manifold = manifolds.Product([
    manifolds.PositiveDefinite(toy_dim + 1, k=toy_components), 
    manifolds.Euclidean(toy_components - 1)
])

# (2) Define cost function
# The parameters must be contained in a list theta.
def cost(theta):
    # Unpack parameters
    nu = np.concatenate([theta[1], [0]], axis=0)
    
    S = theta[0]
    logdetS = np.expand_dims(np.linalg.slogdet(S)[1], 1)
    y = np.concatenate([samples.T, np.ones((1, len(samples)))], axis=0)

    # Calculate log_q
    y = np.expand_dims(y, 0)
    
    # 'Probability' of y belonging to each cluster
    log_q = -0.5 * (np.sum(y * np.linalg.solve(S, y), axis=1) + logdetS)

    alpha = np.exp(nu)
    alpha = alpha / np.sum(alpha)
    alpha = np.expand_dims(alpha, 1)
    
    loglikvec = logsumexp(np.log(alpha) + log_q, axis=0)
    return -np.sum(loglikvec)


problem = opt.Problem(manifold=manifold, cost=cost, verbosity=2)

# (3) Instantiate a Pymanopt solver
solver = solvers.SteepestDescent()

# let Pymanopt do the rest
Xopt = solver.solve(problem)

Once Pymanopt has finished the optimisation we can obtain the inferred parameters as follows:

In [0]:
def extract_gaussian_parameters(Xopt, n=1):
    params, probas = Xopt
    
    mus, sigmas = [], []
    
    for p in params:
        mu = p[0:2,2:3]
        sigma = p[:2, :2] - mu.dot(mu.T)
        mus.append(mu)
        sigmas.append(sigma)
    
    pis = np.exp(np.concatenate([probas, [0]], axis=0))
    pis = pis / np.sum(pis)
    
    return mus, sigmas, pis

In [0]:
toy_mus_opt, toy_sigmas_opt, toy_pis_opt = extract_gaussian_parameters(Xopt, n=3)
toy_mus_opt, toy_sigmas_opt, toy_pis_opt

In [0]:
fig = plt.figure(figsize=(8,8))
ax = fig.gca()

for mu, sigma in zip(toy_mus_opt, toy_sigmas_opt):
    plot_gaussian(mu, sigma, ax)

colors = ['r', 'g', 'b', 'c', 'm']
for i in range(toy_components):
    indices = (i == components)
    ax.scatter(samples[indices, 0], samples[indices, 1], alpha=.4, color=colors[i % toy_components])


And convince ourselves that the inferred parameters are close to the ground truth parameters.

### GMM with real-world data using Riemannian optimization

Certain real-world datasets can be sufficiently closely modelled by the GMM. One instance might be low-dimensional word embeddings. An accompanying notebook `riemannian_opt_text_preprocessing.ipynb` details how these data were obtained. 

In [0]:
import pandas as pd
df = pd.read_csv(DATA_PATH + 'tsne_result_training_part.csv', index_col=0)
df

In [0]:
samples = df[['x', 'y']].values

In [0]:
plt.scatter(samples[:, 0], samples[:, 1], alpha=0.5)

For the optimization to be a little more stable, we standardize the data.

In [0]:
from sklearn.preprocessing import StandardScaler
samples = StandardScaler().fit_transform(samples)

plt.scatter(samples[:, 0], samples[:, 1], alpha=0.5)

Use pretty much the same codes as above, changing the number of components and sample size accordingly.

In [0]:
real_components = 4
real_dim = 2
real_points = len(samples)

In [0]:
import autograd.numpy as np  # import here to avoid errors
from autograd.scipy.misc import logsumexp

# (1) Instantiate the manifold
manifold = manifolds.Product([
    manifolds.PositiveDefinite(real_dim + 1, k=real_components), 
    manifolds.Euclidean(real_components - 1)
])

# (2) Define cost function
# The parameters must be contained in a list theta.
def cost(theta):
    # Unpack parameters
    nu = np.concatenate([theta[1], [0]], axis=0)
    
    S = theta[0]
    logdetS = np.expand_dims(np.linalg.slogdet(S)[1], 1)
    y = np.concatenate([samples.T, np.ones((1, real_points))], axis=0)

    # Calculate log_q
    y = np.expand_dims(y, 0)
    
    # 'Probability' of y belonging to each cluster
    log_q = -0.5 * (np.sum(y * np.linalg.solve(S, y), axis=1) + logdetS)

    alpha = np.exp(nu)
    alpha = alpha / np.sum(alpha)
    alpha = np.expand_dims(alpha, 1)
    
    loglikvec = logsumexp(np.log(alpha) + log_q, axis=0)
    return -np.sum(loglikvec)


problem = opt.Problem(manifold=manifold, cost=cost, verbosity=2)

# (3) Instantiate a Pymanopt solver
solver = solvers.SteepestDescent()

# let Pymanopt do the rest
Xopt = solver.solve(problem)

In [0]:
real_mus_opt, real_sigmas_opt, real_pis_opt = extract_gaussian_parameters(Xopt, n=3)
real_mus_opt, real_sigmas_opt, real_pis_opt

In [0]:
fig = plt.figure(figsize=(8,8))
ax = fig.gca()

for mu, sigma in zip(real_mus_opt, real_sigmas_opt):
    plot_gaussian(mu, sigma, ax)

ax.scatter(samples[:, 0], samples[:, 1], alpha=0.5)

Et voilà – this was a brief demonstration of how to do inference for MoG models by performing Manifold optimisation using Pymanopt.

**TODO HOMEWORK** add riemannian optimization in M-step to speed up EM